## Quick Start

In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="qwen3",
    temperature=0,
    # other params...
)

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
    Extract the desired information from the following passage.

    Only extract the properties mentioned in the 'Classification' function.

    Passage:
    {input}
    """
)

class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")
    translation: str = Field(description="The text translation in English")

# Structured LLM
structured_llm = model.with_structured_output(Classification)

In [3]:
inp = "Je déteste quand je suis obligé de faire quelque chose d'aussi inutile."
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='négatif', aggressiveness=0, language='fr', translation='I hate when I have to do something so useless.')

In [4]:
inp = "Chaque jour est une nouvelle chance de progresser, de sourire et de répandre du positif autour de soi."
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'positive',
 'aggressiveness': 0,
 'language': 'French',
 'translation': 'Every day is a new chance to progress, to smile, and to spread positivity around oneself.'}

## Finer Control

In [5]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["positive", "neutral", "negative"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )
    translation: str = Field(description="The text translation in English")

In [6]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

model = ChatOllama(
    model="qwen3",
    temperature=0,
    # other params...
).with_structured_output(
    Classification
)

In [7]:
inp = "Je déteste quand je suis obligé de faire quelque chose d'aussi inutile."
prompt = tagging_prompt.invoke({"input": inp})
response = model.invoke(prompt)

response

Classification(sentiment='negative', aggressiveness=1, language='french', translation='I hate when I have to do something so useless.')

In [8]:
inp = "Chaque jour est une nouvelle chance de progresser, de sourire et de répandre du positif autour de soi."
prompt = tagging_prompt.invoke({"input": inp})
response = model.invoke(prompt)

response.model_dump()

{'sentiment': 'positive',
 'aggressiveness': 1,
 'language': 'french',
 'translation': 'Every day is a new chance to progress, to smile and to spread positivity around oneself.'}

In [9]:
inp = "Weather is ok here, I can go outside without much more than a coat"
prompt = tagging_prompt.invoke({"input": inp})
model.invoke(prompt)

Classification(sentiment='positive', aggressiveness=1, language='english', translation='The weather is okay here, I can go outside without much more than a coat')